In [ ]:
## import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
# from google.colab import files  ## to export our files to our computer drive
import random  ##iloc/loc

## Let's import the required libaries to create a delay
from random import randrange ##  allows us to randomize numbers library
import time ## time tracker

## Single page scrape

We want to scrape a table that contains NFL player salaries for 2019. 

The webpage is ```https://sandeepmj.github.io/scrape-example-page/"```

In [ ]:
##scrape url website
url = "https://sandeepmj.github.io/scrape-example-page"
page = requests.get(url)
print(page.status_code)  ## should print 200. checks http response code status


In [ ]:
## turn into soup
soup = BeautifulSoup(page.content, "html.parser")
print(type(soup))

In [ ]:
## MUST turn html into a string
html = soup.prettify()
print(type(html))

In [ ]:
## use Pandas to read tables on page
df = pd.read_html(html)
df

In [ ]:
## Do we want the first table?
df[0]

In [ ]:
## Do we want the the second table?
df[1]

In [ ]:
## store it into a copy called nfl_df
fl_df = df[1].copy() 
fl_df

## Multiple page scrape

We want to scrape the tables scattered across this site

https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html



In [ ]:
## How is it different?
url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page{}.html"



### We know we need a placeholder value of upto ```4```

### Let's create a variable called  ```total_pages``` to match number of pages on site.


In [ ]:
## total pages to scrape
total_pages = 5

In [ ]:
total_pages = 5 ## number of pages we want to scrape
df_all = [] ## list that will hold all the dataframes that are produced
for url_number in range(1,total_pages):
    link = url.format(url_number)
    print(url_number)
#     print(f"I'm on page {page}")
    site = requests.get(link)
    try: 
        if site.status_code == 200:
            print(f"got it...scraping page...{link}")
            soup = BeautifulSoup(site.content, "html.parser")
            html = soup.prettify()  ## turn soup into html 
            df = pd.read_html(html) ## turn html table into a df using pandas
            df_all.append(df[0]) ## append that first table to a list
            ## let's not forget to snooze
            snooze = randrange(5,7)
            print(f"snoozing for {snooze} seconds before scraping next link.")
            time.sleep(snooze)

        else:
            print(f"oh no! {link} returned:", site.status_code)
    except: 
        print(f"I can't seem to find these urls")
        
df_all ## what does our list look like

In [ ]:
## FUNCTION to download individual dataframes in a list as a single csv
def combine_tables(list_name,filename):
  '''
  Takes dataframes in a l ist and combines into a single CSV.
  Tables must have identical column headers and order
  Arguments: name of list produced by tabula and the CSV name you want (in quotes as a string)
  '''
  dataframes = [pd.DataFrame(a_table) for a_table in list_name] ## list comprehension to turn each tabula table into a dataframe
  df = pd.concat(dataframes) ## join/concat all the dataframes into one dataframe
  df.reset_index(inplace = True, drop = True) ## reset index, drop what was there before
  df.to_csv(filename, encoding='utf-8', index=False) ## convert that single dataframe into a csv
#   files.download(filename) ## download it
  print(f"{filename} is in your downloads folder!")

In [ ]:
## call that function
combine_tables(df_all,"animals_today.csv")